# openhermes-functions

Demonstrates how to implement function calling at inference time using the "OpenHeremes-2.5-Mistral7B" checkpoint

Source: https://github.com/abacaj/openhermes-function-calling/blob/main/openhermes-functions.ipynb (or https://nbsanity.com/static/f491f7e30f8e9d70dfc72acf9d841afc/openhermes-functions.html)

In [1]:
import gc, inspect, json, re
import xml.etree.ElementTree as ET
from functools import partial
from typing import get_type_hints

import transformers
import torch

from langchain.chains.openai_functions import convert_to_openai_function
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field, validator

In [2]:
model_name = "teknium/OpenHermes-2.5-Mistral-7B"

## Utility Methods

In [3]:
def load_model(model_name: str):
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

    with torch.device("cuda:0"):
        model = transformers.AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).eval()
    
    return tokenizer, model

In [4]:
tokenizer, model = load_model(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:

def delete_model(*args):
    for var in args:
        if var in globals():
            del globals()[var]

    gc.collect()
    torch.cuda.empty_cache()


In [6]:
delete_model("model", "tokenizer")

## Function Calling

### A. Using Python Functions

#### Class/Function Examples

In [7]:
class Article:
    pass

class Weather:
    pass

class Directions:
    pass

def calculate_mortgage_payment(loan_amount: int, interest_rate: float, loan_term: int) -> float:
    """Get the monthly mortgage payment given an interest rate percentage."""
    
    # TODO: you must implement this to actually call it later
    pass

def get_article_details(title: str, authors: list[str], short_summary: str, date_published: str, tags: list[str]) -> Article:
    '''Get article details from unstructured article text.
date_published: formatted as "MM/DD/YYYY"'''
    
    # TODO: you must implement this to actually call it later
    pass

def get_weather(zip_code: str) -> Weather:
    """Get the current weather given a zip code."""
    
    # TODO: you must implement this to actually call it later
    pass

def get_directions(start: str, destination: str) -> Directions:
    """Get directions from Google Directions API.
start: start address as a string including zipcode (if any)
destination: end address as a string including zipcode (if any)"""
    
    # TODO: you must implement this to actually call it later
    pass

#### Serialization Methods

In [8]:
def get_type_name(t):
    name = str(t)
    if "list" in name or "dict" in name:
        return name
    else:
        return t.__name__

In [9]:
print(get_type_name(Article))
print(get_type_name(get_weather))

Article
get_weather


In [10]:
def serialize_function_to_json(func):
    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    function_info = {
        "name": func.__name__,
        "description": func.__doc__,
        "parameters": {
            "type": "object",
            "properties": {}
        },
        "returns": type_hints.get('return', 'void').__name__
    }

    for name, _ in signature.parameters.items():
        param_type = get_type_name(type_hints.get(name, type(None)))
        function_info["parameters"]["properties"][name] = {"type": param_type}

    return json.dumps(function_info, indent=2)

In [11]:
print(serialize_function_to_json(get_article_details))
print(serialize_function_to_json(get_weather))

{
  "name": "get_article_details",
  "description": "Get article details from unstructured article text.\ndate_published: formatted as \"MM/DD/YYYY\"",
  "parameters": {
    "type": "object",
    "properties": {
      "title": {
        "type": "str"
      },
      "authors": {
        "type": "list[str]"
      },
      "short_summary": {
        "type": "str"
      },
      "date_published": {
        "type": "str"
      },
      "tags": {
        "type": "list[str]"
      }
    }
  },
  "returns": "Article"
}
{
  "name": "get_weather",
  "description": "Get the current weather given a zip code.",
  "parameters": {
    "type": "object",
    "properties": {
      "zip_code": {
        "type": "str"
      }
    }
  },
  "returns": "Weather"
}


### B. Using Pydantic

#### Pydantic Examples

In [12]:

class Joke(BaseModel):
    """Get a joke that includes the setup and punchline"""
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field
    
class Actor(BaseModel):
    """Get the films and/or TV shows an actor has appeared in"""
    name: str = Field(description="name of the actor")
    film_names: list[str] = Field(description="list of films the actor appeared in")
    tv_shows: list[str] = Field(description="list of T.V. shows the actor appeared in")

#### Serialization Methods

In [13]:
convert_pydantic_to_openai_function(Actor)

{'name': 'Actor',
 'description': 'Get the films and/or TV shows an actor has appeared in',
 'parameters': {'title': 'Actor',
  'description': 'Get the films and/or TV shows an actor has appeared in',
  'type': 'object',
  'properties': {'name': {'title': 'Name',
    'description': 'name of the actor',
    'type': 'string'},
   'film_names': {'title': 'Film Names',
    'description': 'list of films the actor appeared in',
    'type': 'array',
    'items': {'type': 'string'}},
   'tv_shows': {'title': 'Tv Shows',
    'description': 'list of T.V. shows the actor appeared in',
    'type': 'array',
    'items': {'type': 'string'}}},
  'required': ['name', 'film_names', 'tv_shows']}}

## Inference

In [14]:
def extract_function_calls(completion):
    completion = completion.strip()
    pattern = r"(<multiplefunctions>(.*?)</multiplefunctions>)"
    match = re.search(pattern, completion, re.DOTALL)
    if not match:
        return None
    
    multiplefn = match.group(1)
    root = ET.fromstring(multiplefn)
    functions = root.findall("functioncall")
    return [json.loads(fn.text) for fn in functions]

In [31]:
def generate_hermes(prompt, model, tokenizer, generation_config_overrides={}):
    fn = """{"name": "function_name", "arguments": {"arg_1": "value_1", "arg_2": value_2, ...}}"""
    prompt = f"""<|im_start|>system
You are a helpful assistant with access to the following functions:

{serialize_function_to_json(get_weather)}

{serialize_function_to_json(calculate_mortgage_payment)}

{serialize_function_to_json(get_directions)}

{serialize_function_to_json(get_article_details)}

{convert_pydantic_to_openai_function(Joke)}

{convert_pydantic_to_openai_function(Actor)}

To use these functions respond with:
<multiplefunctions>
    <functioncall> {fn} </functioncall>
    <functioncall> {fn} </functioncall>
    ...
</multiplefunctions>

Edge cases you must handle:
- If there are no functions that match the user request, you will respond politely that you cannot help.<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant"""

    generation_config = model.generation_config
    generation_config.update(
        **{
            **{
                "use_cache": True,
                "do_sample": True,
                "temperature": 0.2,
                "top_p": .1,
                "top_k": 0,
                "max_new_tokens": 512,
                "eos_token_id": tokenizer.eos_token_id,
                "pad_token_id": tokenizer.eos_token_id,
            },
            **generation_config_overrides,
        }
    )

    model = model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    n_tokens = inputs.input_ids.numel()

    with torch.inference_mode():
        generated_tokens = model.generate(**inputs, generation_config=generation_config)

    return tokenizer.decode(
        generated_tokens.squeeze()[n_tokens:], skip_special_tokens=False
    )

### Tests

In [16]:
tokenizer, model = load_model(model_name=model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
%%time

generation_func = partial(generate_hermes, model=model, tokenizer=tokenizer)

prompts = [
    "Tell me a joke about Elon Musk!",
    "What movies and TV shows has Brad Pitt appeared in?",
    "What's the weather in 10001?",
    "Determine the monthly mortgage payment for a loan amount of $200,000, an interest rate of 4%, and a loan term of 30 years.",
    "What's the current exchange rate for USD to EUR?"
]

for prompt in prompts:
    completion = generation_func(prompt)
    functions = extract_function_calls(completion)

    if functions:
        print(functions)
    else:
        print(completion.strip())
    print("="*100)

delete_model("generation_func")

<functioncall> {"name": "Joke", "arguments": {"setup": "Why did Elon Musk go to the doctor?", "punchline": "Because he had a Tesla-cough!"}} </functioncall><|im_end|>
[{'name': 'Actor', 'arguments': {'name': 'Brad Pitt', 'film_names': [], 'tv_shows': []}}]
[{'name': 'get_weather', 'arguments': {'zip_code': '10001'}}]
[{'name': 'calculate_mortgage_payment', 'arguments': {'loan_amount': 200000, 'interest_rate': 0.04, 'loan_term': 30}}]
I'm sorry, but I don't have the functionality to provide exchange rates. Could you please ask me something else?<|im_end|>
CPU times: user 8.65 s, sys: 283 ms, total: 8.93 s
Wall time: 8.92 s


In [33]:
%%time

generation_func = partial(generate_hermes, model=model, tokenizer=tokenizer)

prompts = [
    "Tell me a joke about one of the movies Leonardo Decaprio appears in",
    "What's the weather for 92024?",
    "I'm planning a trip to Killington, Vermont (05751) from Hoboken, NJ (07030). Can you get me weather for both locations and directions?",
    "What's the current exchange rate for USD to EUR?"
]

for prompt in prompts:
    completion = generation_func(prompt)
    functions = extract_function_calls(completion)

    if functions:
        print(functions)
    else:
        print(completion.strip())
    print("="*100)

delete_model("generation_func")

<functioncall> {"name": "Actor", "arguments": {"name": "Leonardo DiCaprio", "film_names": ["Titanic", "The Revenant", "Inception", "The Wolf of Wall Street"]}} </functioncall>

<functioncall> {"name": "Joke", "arguments": {"setup": "Why did the Titanic sink?", "punchline": "Because Leo couldn't handle all that ice!"}} </functioncall><|im_end|>
[{'name': 'get_weather', 'arguments': {'zip_code': '92024'}}]
[{'name': 'get_weather', 'arguments': {'zip_code': '05751'}}, {'name': 'get_weather', 'arguments': {'zip_code': '07030'}}, {'name': 'get_directions', 'arguments': {'start': 'Hoboken, NJ 07030', 'destination': 'Killington, VT 05751'}}]
I'm sorry, but I don't have the functionality to provide exchange rates. Could you please ask me something else?<|im_end|>
CPU times: user 14 s, sys: 262 ms, total: 14.3 s
Wall time: 14.3 s


## Cleanup

In [ ]:
delete_model("model", "tokenizer", "generation_func")